## ISRC Python Workshop 2: Web Scraping (Mar. 09th 2017)

### 1. Use off-the-shelf tools: _Do not reinvent the wheel!!!_

#### Use Application Programming Interfaces (APIs): Twitter API as an example

Many web servers have their own APIs ready to use. By using these convenient tools, we can get started right off following their documentations and examples without any manual efforts. We will be using <a href="https://apps.twitter.com/" target="_blank">Twitter API</a> as an example.

First, we have to register an account for Twitter Developer and register an app. Let's go to https://dev.twitter.com/ and get an app togther. <a href="https://python-twitter.readthedocs.io/en/latest/getting_started.html" target="_b lank">Here</a>'s a quick start on how you can do this.

After we obtain *__consumer key__*, *__consumer secret__*, *__access token__*, and *__access token secret__*, we are ready to retrieve some data from Twitter!

In [ ]:
## suppress warnings
import warnings
warnings.filterwarnings('ignore')

## read my app keys
with open("./twitter_keys.csv", "r") as twitter_keys:
    keys = twitter_keys.read()
    consumer_key, consumer_secret, access_token, access_secret = \
        keys.split("\n")[:-1]

## load twitter package, which a well-written Python package for Twitter APIs
import twitter
api = twitter.Api(consumer_key=consumer_key,
                  consumer_secret=consumer_secret,                  
                  access_token_key=access_token,
                  access_token_secret=access_secret)

## check status
print(api.VerifyCredentials())

In [ ]:
## Try to do some simple tasks
# get statuses
statuses = api.GetUserTimeline(screen_name="Zhiya Zuo")
print([s.text for s in statuses])
statuses = api.GetUserTimeline(user_id="2740697738")
print([s.created_at for s in statuses])

## Get your friends
friends = api.GetFriends()
print([f.name for f in friends])

In [ ]:
## More interestingly, let's go get some tweets from Twitter
## See https://dev.twitter.com/rest/public/search for more informaiton on how to construct a query
results = api.GetSearch(
    raw_query="q=uiowa&result_type=popular&since=2014-12-01&count=20&lang=en")
# How to set `lang` parameter -> https://dev.twitter.com/rest/reference/get/help/languages

# show all the text in the retrieved tweets, with user screen name highlited
from IPython.display import clear_output
for tw in results:
    clear_output() # clear output in iPython Notebook after each print
    print("%s tweeted by \033[41m%s\033[0m"%(tw.text, tw.user.screen_name))

### 2. Create your own manual parsing programs 

#### Manually scraping

Sometimes, they have APIs but they have no well-written packages in the language you prefer (e.g. only Java but no Python libraries). Even worse, there may not be APIs for the public and we have to design a scraper to retrieve all the relevant informaiton we want. In such cases, we can manually look at the returned values and build our own wrapper functions.

#### Preliminiary examples

Examples from <a href="https://www.w3schools.com/html/tryit.asp?filename=tryhtml_basic_document" target="blank_">w3schools</a>.

```html
<!DOCTYPE html>
<html>
<body>

<h1>My First Heading</h1>

<p>My 1st paragraph.</p>
<p>My 2nd paragraph.</p>
<p>My 3rd paragraph.</p>

</body>
</html>
```

Save this code to your disk as `sample.html` (or any other name). We will use a great library called ___`Beautiful Soup`___ to read the contents from Python. You may also need to install lxml, which is for parsing specific formats (e.g., html and xml).

In [ ]:
## Do the following if you have not
# pip install beautifulsoup4 lxml

from bs4 import BeautifulSoup as Soup

with open("./sample.html", "r") as sample:
    sample_contents = sample.read()

sample_soup = Soup(sample_contents)
# by printing it, we can see the exact contents as shown above with proper indentation
print(sample_soup.prettify())

In [ ]:
## Get the contents of interest: all the p's
# ** p means paragraph in html. Check more tag definitions on w3schools.org
p_tags = sample_soup.find_all("p")
for p in p_tags:
    print(p.text)

#### Locate your contents of interest in browser
As you can see, this is very straightforward. Let's use a real website for illustration. For example, if we are interested in company profiles, we can scrape from Google Finance. We will be using <a href="https://www.google.com/finance?q=NYSE%3AIBM&ei=Ij62WPHgGdSLmAHrja_wAQ" target="_blank">IBM's profile</a> as an example. However, you may find off-the-shelf packages. We will only use this for an introduction on how to scrape manually.

To view the "text style" or the real structure of a web page, you can use ___`developer tools`___ function in your browser. For example, you can see something like this. If you move your mouse to a place, the console will show you the corresponding tags in the source html files. You will find that the description text is located within a `p` tag.

<img src="http://i.imgur.com/IEl2uyG.png" width="1000">

In [ ]:
import urllib2 
ibm_url = "https://www.google.com/finance?q=NYSE%3AIBM&ei=Ij62WPHgGdSLmAHrja_wAQ"
## read it as strings
ibm_page = urllib2.urlopen(ibm_url).read()
## convert it to a soup object
ibm_soup = Soup(ibm_page)
## find the correponding tag. Note that class_ has a trailing underscore
summary_tag = ibm_soup.find("div", class_="companySummary")
print("\033[43m%s\033[0m"%(summary_tag.text))

With this in mind, you can scrape almost any webpage of interest. Other formats such as <a href="http://www.json.org/" target="_blank">JSON</a> and <a href="https://www.w3.org/XML/" target="_blank">XML</a> do have high similarities and a few differences. They are not very difficult to know the basics!

***But keep in mind that you should act politely, with propoer permission. To find out whether specific paths/contents are allowed to be scraped, you can check their ___`robots.txt`___. For example, <a href="https://www.google.com/robots.txt" target="_blank">here's</a> the permission information set by Google.***

### 3. Exercise: build our own Scopus scraper

We will be using <a href="https://dev.elsevier.com/" target="_blank">Scopus APIs</a> as an example (it seems like they are actively building such tools to facilitate developing applications but we can still DIY wrappers that most fit our own usage). I build my own called <a href="http://zhiyzuo.github.io/python-scopus/" target="_blank">python-scopus</a>.

#### A quick start

The first step will still be getting your API key for access to Elsevier's database. Go to https://dev.elsevier.com/ and request a key. Then we can write a simple query to try it out!

In [ ]:
with open("./scopus_apikey.csv", "r") as scopus_key:
    key = scopus_key.read().strip("\n")
## check out http://api.elsevier.com/documentation/SCOPUSSearchAPI.wadl
## search tips: http://api.elsevier.com/documentation/search/SCOPUSSearchTips.htm
search_uri = "http://api.elsevier.com/content/search/scopus"

import requests
## notice how we supply the query parameters using requests package
## http://docs.python-requests.org/en/master/user/quickstart/#make-a-request
par = {'apikey': key, 'query': 'TITLE-ABS-KEY(protein structure)'}
r = requests.get(search_uri, params=par)
## click to see
print("XML: %s"%r.url)

## by default this is an XML repsonse, we can ask elsevier to return a JSON object
par_json = {'apikey': key, 'query': 'TITLE-ABS-KEY(protein structure)', 'httpAccept':'application/json'}
r_json = requests.get(search_uri, params=par_json)
## click to see
print("JSON: %s"%r_json.url)

For parsing the JSON object, you can use <a href="http://docs.python-guide.org/en/latest/scenarios/json/" target="_blank">***`JSON`***</a> library, which we will cover later. 

So, we currently have something like this:

<img src="http://i.imgur.com/4NnM00t.png">

We can actually use the same trick as we did for the html files for xml formats: first download it using ___`urlopen`___ function in ___`urllib2`___ library. However, ___`requests`___ make all these much easier for us:

In [ ]:
## a method to convert the response into JSON!
resp = r.json()
print(resp)

In [ ]:
## by default it will return 25 results.
print("Showing %s items."%len(resp['search-results']['entry']))
## use start index to iterate thru all results
total_count = resp['search-results']['opensearch:totalResults']
start_index = resp['search-results']['opensearch:startIndex']
print("A total number of %s results found in Scopus; Start index is %s."%(total_count, start_index))
## Use a dictionary-style to access the values.
print("\033[43m%s\033[0m"%resp['search-results']['entry'][0])

Suppose we only want the titles, we can write a very simple function to build a wrapper for code reuse:

In [ ]:
def search_scopus(key, query, index=0, verbose=True):
    '''
        Search Scopus database using key as api key, with query.
        
        Parameters
        ----------
        key : string
            Elsevier api key. Get it here: https://dev.elsevier.com/index.html
        query : string
            Search query. See more details here: http://api.elsevier.com/documentation/search/SCOPUSSearchTips.htm
        index : int
            Start index. Will be used in search_scopus_plus function
        verbose : bool
            Verbose mode.
        
        Returns
        -------
        pandas DataFrame
            id column stores scopus id and title column stores titles
    '''
    
    import requests
    par = {'apikey': key, 'query': 'TITLE-ABS-KEY(protein structure)', 'start': index}
    r = requests.get(search_uri, params=par)
    resp = r.json()
    ## print out some summaries
    total_count = resp['search-results']['opensearch:totalResults']
    start_index = resp['search-results']['opensearch:startIndex']
    entries = resp['search-results']['entry']
    if verbose:
        print("Going to: %s"%r.url)
        print("A total number of %s results found in Scopus; Showing %d; Start index is %s."%\
              (total_count, len(entries), start_index))
        print("--------------------------------------")  

    # iterate thru each entry
    # You can also do this in a list comprehension with one line of code
    # (I did not test the following code but it should work with minimal modification)
    # id_title_list = [(entry['dc:identifier'].split(":")[-1], entry["dc:title"]) for entry in entries]
    id_title_list = []
    for entry in entries:
        scopus_id = entry['dc:identifier'].split(":")[-1]
        id_title_list.append((scopus_id, entry["dc:title"]))
    
    ## use pd.DataFrame to better store the results
    import pandas as pd
    result_df = pd.DataFrame(id_title_list, columns=list(("id", "title")))
    return(result_df)

In [ ]:
result_df = search_scopus(key, 'TITLE-ABS-KEY(protein structure)')
# use iPython's functions to print data frames prettier
from IPython.display import display, HTML
display(result_df)
# OR: HTML(result_df.to_html())

## * Conclusion

Hopefully you see how the two different types of web data retrieval work and you will be able to scrape your own data now! Note that in our last example of scraping Scopus data, we only get 25 results. By a simple tweak of varying the indices, we can retrieve all of them! The following is a revised version of ***`search_scopus`*** to get all entries, just for your references, which is part of my code for ***`python-scopus`***.

In [ ]:
def search_scopus_plus(key, query, number=200):
    '''
        Search Scopus database using key as api key, with query.
        Reuse function ssearch_scopus(key, query, index, verbose)
        
        Parameters
        ----------
        key : string
            Elsevier api key. Get it here: https://dev.elsevier.com/index.html
        query : string
            Search query. See more details here: http://api.elsevier.com/documentation/search/SCOPUSSearchTips.htm
        number : int
            The number of entries to return. By default it is 200.
        
        Returns
        -------
        pandas DataFrame
            id column stores scopus id and title column stores titles
    '''
    
    if type(number) is not int or number > total_count:
        import sys
        raise ValueError("%s is not a valid input for the number of entries to return." %number)
    
    results_df = search_scopus(key, query, verbose=False)
    if number < 25:
        # if less than 25, just one page of response is enough
        return results_df[:number]
    
    # if larger than, go to next few pages until enough
    index = 1
    while True:
        results_df = results_df.append(search_scopus(key, query, index, False), ignore_index=True)
        if len(results_df) >= number:
            return results_df[:number]
        index += 1

In [ ]:
result_df_20 = search_scopus_plus(key, 'TITLE-ABS-KEY(protein structure)',20)
display(result_df_20)

In [ ]:
result_df_40 = search_scopus_plus(key, 'TITLE-ABS-KEY(protein structure)',40)
display(result_df_40)